## T5(Text-toText Transfer Transformer)

기존 자연어 처리 모델은 대부분 입력 문장을 벡터나 행렬로 변환한 뒤, 이를 통해 출력 문장ㅇ르 생성하는 방식이거나, 출력값이 클래스나 입력값의 일부를 반환하는 형식으로 동작.

T5는 출력을 모두 토큰 시퀀스로 처리하는 Text to Text structure.

입력과 출력의 형태를 자유로이 다룰 수 있으며, 구조상 유연성과 확장성이 뛰어남.

In [1]:
import numpy as np
from datasets import load_dataset

In [2]:
news = load_dataset('argilla/news-summary', split='test')
df = news.to_pandas().sample(5000, random_state=42)[['text', 'prediction']]
df['text'] = 'summarize: ' + df['text']
df['prediction'] = df['prediction'].map(lambda x: x[0]['text'])
train, valid, test = np.split(
    df.sample(frac = 1, random_state = 42), [int(0.6*len(df)), int(0.8*len(df))]
)

C:\Users\dohyeong\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [3]:
train

,text,prediction
9209,"summarize: DANANG, Vietnam (Reuters) - Russian...",Putin says had useful interaction with Trump a...
13236,summarize: NEW YORK (Reuters) - A showdown bet...,NY mayor criticizes Trump's closing public atr...
7828,summarize: (This January 3 story was correcte...,Oil business seen in strong position as Trump ...
18839,summarize: NEW YORK (Reuters) - Washington sta...,Courts likely to probe Trump's intent in issui...
19844,summarize: WASHINGTON (Reuters) - Kristie Kenn...,Kristie Kenney named State Department's new co...
...,...,...
7920,summarize: MOSCOW (Reuters) - President Vladim...,Putin warns North Korea situation on verge of ...
751,"summarize: DANANG, Vietnam (Reuters) - It is n...",New Zealand says unclear if TPP agreement can ...
16847,"summarize: CORALVILLE, Iowa (Reuters) - U.S. R...",Republican candidate Rubio: Fed needs clear ru...
1037,summarize: WASHINGTON (Reuters) - It would not...,Germany's Schaeuble presses ECB to unwind loos...


In [5]:
train['text'][9209]

'summarize: DANANG, Vietnam (Reuters) - Russian President Vladimir Putin said on Saturday he had a normal dialogue with U.S. leader Donald Trump at a summit in Vietnam, and described Trump as civil, well-educated, and comfortable to deal with. Putin said that a mooted bilateral sit-down meeting with Trump did not happen at the Asia-Pacific Economic Cooperation summit, citing scheduling issues on both sides and unspecified protocol issues. Putin, at a briefing for reporters at the end of the summit, said there was still a need for further U.S.-Russia contacts, both at the level of heads of state and their officials, to discuss issues including security and economic development.   '

In [6]:
train['prediction'][9209]

'Putin says had useful interaction with Trump at Vietnam summit'